<a href="https://colab.research.google.com/github/jambrizz/CSE-450/blob/main/group_three_a_starter_bikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import all libraries here, and only here


import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.layers import Normalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error


In [ ]:
#Make all changes to the data here, and only here
bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')


#One hot encode weather
weather_encoded = pd.get_dummies(bikes['weathersit'], prefix='weather')


bikes = pd.concat([bikes, weather_encoded], axis=1)


bikes.drop('weathersit', axis=1, inplace=True)

season_encoded = pd.get_dummies(bikes['season'], prefix='season')


bikes = pd.concat([bikes, season_encoded], axis=1)


bikes.drop('season', axis=1, inplace=True)

bikes['year'] = pd.to_datetime(bikes['dteday'], format='%m/%d/%Y').dt.year

bikes['month'] = pd.to_datetime(bikes['dteday'], format='%m/%d/%Y').dt.month


month_encoded = pd.get_dummies(bikes['month'], prefix='month')


bikes = pd.concat([bikes, month_encoded], axis=1)


bikes.drop('month', axis=1, inplace=True)



bikes['day'] = pd.to_datetime(bikes['dteday'], format='%m/%d/%Y').dt.day

bikes['day_category'] = pd.cut(
    bikes['day'],
    bins=[0, 10, 20, 31],  # Defines ranges: (0-10], (10-20], (20-31]
    labels=['beginning', 'middle', 'end']
)




bikes = pd.get_dummies(bikes, columns=['day_category'], drop_first=False)
bikes.drop(columns=['day'], inplace=True)





bikes.drop('dteday',axis=1,inplace=True)


#Setting up the training sets, don't modify any columns after this point, do everything before

bikes['total users'] = bikes['casual'] + bikes['registered']



december_2023_data = bikes[(bikes['year'] == 2023) & (bikes['month_12'] == 1)]

# The remaining data (excluding December 2023) will be used for training
train_data = bikes[~((bikes['year'] == 2023) & (bikes['month_12'] == 1))]

dec_test = train_data.drop(columns=['total users'])
dec_testy = train_data['total users']


dec_test.drop('casual',axis=1,inplace=True)
dec_test.drop('registered',axis=1,inplace=True)




#y = bikes[['casual','registered']]

bikes.drop('casual',axis=1,inplace=True)
bikes.drop('registered',axis=1,inplace=True)
X = bikes.drop('total users', axis=1,inplace=False)

y = bikes['total users']
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = .8, random_state=42)



bikes.info()
X_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112475 entries, 0 to 112474
Data columns (total 32 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   hr                      112475 non-null  float64
 1   temp_c                  112475 non-null  float64
 2   feels_like_c            112475 non-null  float64
 3   hum                     112475 non-null  float64
 4   windspeed               112475 non-null  float64
 5   holiday                 112475 non-null  int64  
 6   workingday              112475 non-null  int64  
 7   weather_1               112475 non-null  bool   
 8   weather_2               112475 non-null  bool   
 9   weather_3               112475 non-null  bool   
 10  weather_4               112475 non-null  bool   
 11  season_1                112475 non-null  bool   
 12  season_2                112475 non-null  bool   
 13  season_3                112475 non-null  bool   
 14  season_4            

,hr,temp_c,feels_like_c,hum,windspeed,holiday,workingday,weather_1,weather_2,weather_3,...,month_6,month_7,month_8,month_9,month_10,month_11,month_12,day_category_beginning,day_category_middle,day_category_end
22023,18.0,32.9,36.2,0.5064,20.0,0,0,True,False,False,...,False,True,False,False,False,False,False,True,False,False
30281,21.0,23.4,23.4,0.7048,8.6,0,0,True,False,False,...,True,False,False,False,False,False,False,False,True,False
83263,17.0,29.4,31.6,0.5952,14.8,0,1,True,False,False,...,False,True,False,False,False,False,False,True,False,False
63147,11.0,3.3,-2.7,0.3112,37.4,0,1,True,False,False,...,False,False,False,False,False,False,False,False,True,False
18592,18.0,4.1,0.0,0.8514,19.3,0,1,False,False,True,...,False,False,False,False,False,False,False,False,True,False


In [ ]:
#Here is where the model magic happens



best_r_score = 0
best_model = None

#Early Stopping setup
X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
early_stopping = EarlyStopping(monitor='val_loss',   # Monitor the validation loss
                               patience=10,           # Number of epochs with no improvement to wait
                               restore_best_weights=True)

#Data normalization
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(X_train2))



#Training the model


for i in range(1,2):

  bike_model = tf.keras.Sequential([
    normalizer,

    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),

    layers.Dense(1)
])
  bike_model.compile(loss = tf.keras.losses.MeanSquaredError(), optimizer = tf.keras.optimizers.Adam())
  bike_model.fit(X_train2, y_train2, epochs=100, batch_size = i * 32, callbacks = [early_stopping], validation_data =(X_val,y_val))


  #Here is where the prediction is given the r2 score
  predictions = bike_model.predict(X_test)

  r_score = r2_score(y_test, predictions)

  print(r_score)
  if r_score > best_r_score:
    best_r_score = r_score
    best_model = bike_model
    besti = i




print(best_r_score)

print(besti)









Epoch 1/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 147761.5312 - val_loss: 23609.0371
Epoch 2/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 34531.0117 - val_loss: 18047.4746
Epoch 3/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - loss: 30231.0215 - val_loss: 15203.6221
Epoch 4/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - loss: 25355.1465 - val_loss: 15243.0693
Epoch 5/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - loss: 24317.9844 - val_loss: 13295.0732
Epoch 6/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - loss: 21371.2188 - val_loss: 10653.4326
Epoch 7/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - loss: 20161.0332 - val_loss: 10941.4297
Epoch 8/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - loss: 19053.6289 - val_loss: 9282.4570
Epoch 9/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 18556.1035 - val_loss: 9868.6162
Epoch 10/100
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - loss: 17625.5645 - val_loss: 8762.1270
Epoch 11/100
2250/225

In [ ]:
# I am testing out how it does on december here as it struggles with Christmas time specifically


predicDec = best_model.predict(dec_test)
print(r2_score(dec_testy,predicDec))




3515/3515 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step
0.9502970576286316


In [ ]:
from google.colab import files
dec_testy.to_csv('team3-module4-december-2023.csv', index=False)
files.download('team3-module4-december-2023.csv')
dec_testy.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,total users
0,16
1,38
2,31
3,12
4,1


In [ ]:
#Don't do this unless it isn't overfitting yet

prev_best = best_model




best_model.fit(X_train2, y_train2, epochs=30, batch_size = 32, callbacks = [early_stopping], validation_data =(X_val,y_val))


predictions = best_model.predict(X_test)
r_score = r2_score(y_test, predictions)
print(r_score)

Epoch 1/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 4711.7715 - val_loss: 6429.6509
Epoch 2/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 4666.3853 - val_loss: 6622.3975
Epoch 3/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 4648.8330 - val_loss: 6301.4443
Epoch 4/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 4700.7964 - val_loss: 6717.0718
Epoch 5/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 4644.2764 - val_loss: 7040.6553
Epoch 6/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 4521.1470 - val_loss: 6430.9263
Epoch 7/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 4552.4995 - val_loss: 6567.4766
Epoch 8/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 4573.2866 - val_loss: 6248.8623
Epoch 9/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 4388.8955 - val_loss: 6525.7861
Epoch 10/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 4383.1494 - val_loss: 6712.0225
Epoch 11/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/ste

In [ ]:
print(best_r_score)
print(r_score)

if r_score > best_r_score:
  best_r_score = r_score

print(mean_squared_error(y_test, predictions))

print(np.sqrt(mean_squared_error(y_test, predictions)))

0.9465712308883667
0.9465712308883667
6281.09716796875
79.2533732276977


In [ ]:
#Data for hold out set

bikes_test = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes_december.csv')

print(bikes_test.shape)
#One hot encode weather
weather_encoded = pd.get_dummies(bikes_test['weathersit'], prefix='weather')


bikes_test = pd.concat([bikes_test, weather_encoded], axis=1)


bikes_test.drop('weathersit', axis=1, inplace=True)

season_encoded = pd.get_dummies(bikes_test['season'], prefix='season')
bikes_test['season_2'] = False
bikes_test['season_3'] = False
bikes_test = pd.concat([bikes_test, season_encoded], axis=1)


bikes_test.drop('season', axis=1, inplace=True)

bikes_test['year'] = pd.to_datetime(bikes_test['dteday'], format='%m/%d/%Y').dt.year
bikes_test['month'] = pd.to_datetime(bikes_test['dteday'], format='%m/%d/%Y').dt.month


month_encoded = pd.get_dummies(bikes_test['month'], prefix='month')


bikes_test = pd.concat([bikes_test, month_encoded], axis=1)


bikes_test.drop('month', axis=1, inplace=True)


bikes_test['day'] = pd.to_datetime(bikes_test['dteday'], format='%m/%d/%Y').dt.day

bikes_test['day_category'] = pd.cut(
    bikes_test['day'],
    bins=[0, 10, 20, 31],  # Defines ranges: (0-10], (10-20], (20-31]
    labels=['beginning', 'middle', 'end']
)




bikes_test = pd.get_dummies(bikes_test, columns=['day_category'], drop_first=False)
bikes_test.drop(columns=['day'], inplace=True)



for i in range(1, 11):
    bikes_test['month_' + str(i)] = False




bikes_test.drop('dteday',axis=1,inplace=True)
bikes_test.head()
bikes.info()

(1465, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112475 entries, 0 to 112474
Data columns (total 32 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   hr                      112475 non-null  float64
 1   temp_c                  112475 non-null  float64
 2   feels_like_c            112475 non-null  float64
 3   hum                     112475 non-null  float64
 4   windspeed               112475 non-null  float64
 5   holiday                 112475 non-null  int64  
 6   workingday              112475 non-null  int64  
 7   weather_1               112475 non-null  bool   
 8   weather_2               112475 non-null  bool   
 9   weather_3               112475 non-null  bool   
 10  weather_4               112475 non-null  bool   
 11  season_1                112475 non-null  bool   
 12  season_2                112475 non-null  bool   
 13  season_3                112475 non-null  bool   
 14  season_4 

In [ ]:
X_train.head()

,hr,temp_c,feels_like_c,hum,windspeed,holiday,workingday,weather_1,weather_2,weather_3,...,month_6,month_7,month_8,month_9,month_10,month_11,month_12,day_category_beginning,day_category_middle,day_category_end
22023,18.0,32.9,36.2,0.5064,20.0,0,0,True,False,False,...,False,True,False,False,False,False,False,True,False,False
30281,21.0,23.4,23.4,0.7048,8.6,0,0,True,False,False,...,True,False,False,False,False,False,False,False,True,False
83263,17.0,29.4,31.6,0.5952,14.8,0,1,True,False,False,...,False,True,False,False,False,False,False,True,False,False
63147,11.0,3.3,-2.7,0.3112,37.4,0,1,True,False,False,...,False,False,False,False,False,False,False,False,True,False
18592,18.0,4.1,0.0,0.8514,19.3,0,1,False,False,True,...,False,False,False,False,False,False,False,False,True,False


In [ ]:
predictions = best_model.predict(bikes_test)
df = pd.DataFrame()
df['predictions'] = pd.DataFrame(predictions)
df = df[['predictions']]
df.shape

from google.colab import files

df.to_csv('team3-module4' + str(best_r_score) + '-predictions.csv', index=False)
files.download('team3-module4' + str(best_r_score) + '-predictions.csv')
df.head()

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,predictions
0,50.321518
1,42.676971
2,33.901402
3,28.215702
4,19.575573
